# Import Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Load Dataset

In [3]:
url = 'https://drive.google.com/uc?id=1iD6k9BPPpJ2nP84l11i3uPbp5Lz7mWQr'
df = pd.read_csv(url)

# Menampilkan dataframe sebagai tabel
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,a93920d8-e94b-44c1-a5b3-288e18bc8ee7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,perasaan dl nonton tv lokal gratis! skrg Bayar,1,0,NaN,2025-08-30 06:34:59,Hai kak 👋 ada yang spesial dari Vision+ khusus...,2025-08-30 07:13:43,NaN
1,f51f2ae1-85ad-42de-834b-680b2aa513a4,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi terbaik pokokknya, jadi ga ketinggala...",5,0,NaN,2025-08-30 03:23:20,Hi Kak.. Terimakasih atas review-nya.. Selamat...,2025-08-30 04:18:05,NaN
2,1e4bd6d1-8a57-450a-86bd-96c39983484f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Saya sekali berlangganan, tapi saldo dana saya...",1,0,NaN,2025-08-29 19:13:34,Hai kak 👋 ada yang spesial dari Vision+ khusus...,2025-08-30 02:32:54,NaN
3,a106bd81-a19d-4908-8ad5-b814e09d2a5b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"cukup bermanfaat, bs di connect ke decoder di ...",5,0,NaN,2025-08-29 13:46:03,Hi Kak.. Terimakasih atas review-nya.. Selamat...,2025-08-29 13:51:59,NaN
4,3eb38928-3afd-4e12-b26c-cc86f17317a3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"seru, bisa nonton dimana saja",5,0,11.5.1(5)_prd vision+ ahh,2025-08-29 13:45:38,Hi Kak.. Terimakasih atas review-nya.. Selamat...,2025-08-29 13:51:54,11.5.1(5)_prd vision+ ahh
...,...,...,...,...,...,...,...,...,...,...,...
49995,f196214b-b365-4e41-a6f1-d5bae842bb79,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kenapa klo lagi maen bola indosiar nya malah n...,1,0,4.0.2,2019-07-16 12:20:37,"Terima kasih reviewnya, untuk beberapa tayanga...",2019-07-17 01:44:40,4.0.2
49996,d18eccb6-1ac3-4031-b255-e3ea2aa1aeb5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,ok,5,0,NaN,2019-07-16 12:19:32,"Terima kasih atas review-nya, selamat menikmat...",2019-07-17 01:43:42,NaN
49997,85b0b7f8-813e-46b2-ae41-9ca3471690f9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Memuaskan,5,0,NaN,2019-07-16 12:00:51,"Terima kasih atas review-nya, selamat menikmat...",2019-07-17 01:40:33,NaN
49998,296cf91c-d505-4d42-b86f-faed179d1d0b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,gak bisa muter filmnya ... gagal/eror terus.. ...,2,0,NaN,2019-07-16 11:57:38,Mohon Maaf atas ketidaknyamanannya.Silakan cob...,2019-07-17 01:43:21,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              50000 non-null  object
 1   userName              50000 non-null  object
 2   userImage             50000 non-null  object
 3   content               50000 non-null  object
 4   score                 50000 non-null  int64 
 5   thumbsUpCount         50000 non-null  int64 
 6   reviewCreatedVersion  36089 non-null  object
 7   at                    50000 non-null  object
 8   replyContent          41258 non-null  object
 9   repliedAt             41258 non-null  object
 10  appVersion            36089 non-null  object
dtypes: int64(2), object(9)
memory usage: 4.2+ MB


In [5]:
df.isna().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,13911
at,0
replyContent,8742
repliedAt,8742


In [6]:
df = df.drop(columns=['reviewId', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion', 'replyContent',
                      'repliedAt', 'appVersion'])
print(df.columns)

Index(['userName', 'content', 'score', 'at'], dtype='object')


In [7]:
df.head()

,userName,content,score,at
0,Pengguna Google,perasaan dl nonton tv lokal gratis! skrg Bayar,1,2025-08-30 06:34:59
1,Pengguna Google,"Aplikasi terbaik pokokknya, jadi ga ketinggala...",5,2025-08-30 03:23:20
2,Pengguna Google,"Saya sekali berlangganan, tapi saldo dana saya...",1,2025-08-29 19:13:34
3,Pengguna Google,"cukup bermanfaat, bs di connect ke decoder di ...",5,2025-08-29 13:46:03
4,Pengguna Google,"seru, bisa nonton dimana saja",5,2025-08-29 13:45:38


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  50000 non-null  object
 1   content   50000 non-null  object
 2   score     50000 non-null  int64 
 3   at        50000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.5+ MB


# Pre Processing

In [9]:
df.head()

,userName,content,score,at
0,Pengguna Google,perasaan dl nonton tv lokal gratis! skrg Bayar,1,2025-08-30 06:34:59
1,Pengguna Google,"Aplikasi terbaik pokokknya, jadi ga ketinggala...",5,2025-08-30 03:23:20
2,Pengguna Google,"Saya sekali berlangganan, tapi saldo dana saya...",1,2025-08-29 19:13:34
3,Pengguna Google,"cukup bermanfaat, bs di connect ke decoder di ...",5,2025-08-29 13:46:03
4,Pengguna Google,"seru, bisa nonton dimana saja",5,2025-08-29 13:45:38


In [10]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [11]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [12]:
df['text_clean'] = df['content'].apply(cleaningText)

df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)

df['text_slangwords'] = df['text_casefoldingText'].apply(fix_slangwords)

df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)

df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)

df['text_akhir'] = df['text_stopword'].apply(toSentence)

# Labeling

In [13]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [14]:
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [15]:
  results = df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
  results = list(zip(*results))
  df['polarity_score'] = results[0]
  df['polarity'] = results[1]
  print(df['polarity'].value_counts())

polarity
positive    27745
negative    22255
Name: count, dtype: int64


# Data Splitting dan Ekstraksi Fitur

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = df['text_akhir']
y = df['polarity']

tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    min_df=3,
    max_df=0.9,
    sublinear_tf=True
)
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
  )

# Modeling

In [17]:
# Modeling - Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random_forest = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

random_forest.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_train, y_pred_train_rf)
accuracy_test_rf = accuracy_score(y_test, y_pred_test_rf)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.999625
Random Forest - accuracy_test: 0.8993


In [18]:
# Modeling - Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logistic Regression dengan solver dan setting yang cocok untuk teks high-dimensional
logistic_regression = LogisticRegression(
    max_iter=3000,
    solver="saga",
    multi_class="multinomial",
    n_jobs=-1,
    class_weight="balanced"
)

# Latih model
logistic_regression.fit(X_train, y_train)

# Prediksi
y_pred_train_lr = logistic_regression.predict(X_train)
y_pred_test_lr  = logistic_regression.predict(X_test)

# Evaluasi
accuracy_train_lr = accuracy_score(y_train, y_pred_train_lr)
accuracy_test_lr  = accuracy_score(y_test,  y_pred_test_lr)

print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression - accuracy_train: 0.97295
Logistic Regression - accuracy_test: 0.9373


In [19]:
# Modeling - Linear SVM
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

svm_model = LinearSVC(
    dual=False,
    random_state=42
)

svm_model.fit(X_train, y_train)

y_pred_train_svm = svm_model.predict(X_train)
y_pred_test_svm  = svm_model.predict(X_test)

accuracy_train_svm = accuracy_score(y_train, y_pred_train_svm)
accuracy_test_svm  = accuracy_score(y_test,  y_pred_test_svm)

print('LinearSVC - accuracy_train:', accuracy_train_svm)
print('LinearSVC - accuracy_test:',  accuracy_test_svm)

LinearSVC - accuracy_train: 0.995175
LinearSVC - accuracy_test: 0.946
